# ЗАПУСК ИНФЕРЕНСА
Надо положить в папку "files for inference" желаемые фотки для обработки. Результат будет в "results of inference"

In [1]:
import os.path as osp
import glob
import cv2
import numpy as np
import torch
import architecture as arch
from tqdm.auto import tqdm

model_path = 'ESRGAN.pth'  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

test_img_folder = 'files for inference/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)


Using device: cuda


In [2]:
idx = 0
for path in tqdm(glob.glob(test_img_folder)):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('results of inference/{:s}_better.png'.format(base), output)
print('Обработка завершена')

  0%|          | 0/2 [00:00<?, ?it/s]

Обработка завершена
